In [7]:
from scipy.stats import mode
import numpy as np
#from mnist import MNIST
from time import time
import pandas as pd
import os
import matplotlib.pyplot as matplot
import matplotlib
%matplotlib inline

import random
matplot.rcdefaults()
from time import time
from IPython.display import display, HTML
from itertools import chain
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sb
from sklearn.model_selection import ParameterGrid
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
from keras.datasets import mnist
from matplotlib import pyplot
# load dataset
(trainX, trainy), (X_test, Y_test) = mnist.load_data()
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (X_test.shape, Y_test.shape))

Train: X=(60000, 28, 28), y=(60000,)
Test: X=(10000, 28, 28), y=(10000,)


In [3]:

mlp = MLPClassifier(hidden_layer_sizes=(100, ), 
                    max_iter=480, alpha=1e-4,
                    solver='sgd', verbose=10, 
                    tol=1e-4, random_state=1,
                    learning_rate_init=.1)

print(trainX.shape, trainy.shape)

nsamples, nx, ny = trainX.shape
d2_train_dataset = trainX.reshape((nsamples,nx*ny))

mlp.fit(d2_train_dataset, trainy)
print("Training set score: %f" % mlp.score(d2_train_dataset, trainy))

nsamples, nx, ny = X_test.shape
d2_test_dataset = X_test.reshape((nsamples,nx*ny))

print("Test set score: %f" % mlp.score(d2_test_dataset, Y_test))

(60000, 28, 28) (60000,)
Iteration 1, loss = 24643.68645170
Iteration 2, loss = 26149.49418070
Iteration 3, loss = 26141.66292027
Iteration 4, loss = 26133.82218352
Iteration 5, loss = 26125.98384606
Iteration 6, loss = 26118.14789477
Iteration 7, loss = 26110.31425173
Iteration 8, loss = 26102.48314216
Iteration 9, loss = 26094.65405128
Iteration 10, loss = 26086.82758765
Iteration 11, loss = 26079.00336595
Iteration 12, loss = 26071.18158552
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Training set score: 0.112383
Test set score: 0.113400


In [8]:
print(accuracy_score(Y_test, mlp.predict(d2_test_dataset)))
confusion_matrix(mlp.predict(d2_test_dataset), Y_test)
print(classification_report(mlp.predict(d2_test_dataset), Y_test))

0.1134
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.11      0.20      9999
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.11     10000
   macro avg       0.10      0.01      0.02     10000
weighted avg       1.00      0.11      0.20     10000



(a) Find a good combination of parameter values for the MLPClassifier that provides the best accuracy on the 10,000 test images. Describe your parameter choices and why you believe these values are good choices.

We choose Alpha and Max_iter as the parameter to run the model on and select the best from those.

According to Scikit Learn- MLP classfier documentation,
Alpha is L2 or ridge penalty (regularization term) parameter.
Max_iter is Maximum number of iterations, the solver iterates until convergence.
So, these ones makes sense as we could see some changing values of accuracy while tuning them,

In [ ]:
i = 0
df = pd.DataFrame(columns = ['alpha','max_iter','train_acc','test_acc','train_time'])
for a in [0.00001,0.0001,0.001,0.01, 0.1, 1, 10]:
    for mi in [10,100,200,500,1000,2000]:
        st = time()
        mlp = MLPClassifier(alpha=a, max_iter=mi)
        mlp.fit(d2_train_dataset, trainy)
        end = time() - st
        
        acc_tr = accuracy_score(trainy, mlp.predict(d2_train_dataset)) # Train Accuracy
        acc = accuracy_score(Y_test, mlp.predict(d2_test_dataset)) # Test Accuracy
        df.loc[i] = [a,mi,acc_tr,acc,end]
        i=i+1

In [ ]:
df # Results

In [ ]:
acc = []
acc_tr = []
timelog = []
for l in [10,20,50,100,200,500,1000]:
    t = time()
    mlp = MLPClassifier(alpha=0.1, max_iter=200, hidden_layer_sizes=(l,))
    mlp.fit(d2_train_dataset, trainy)
    endt = time() - t
        
    a_tr = accuracy_score(trainy, mlp.predict(d2_train_dataset)) # Train Accuracy
    a = accuracy_score(Y_test, mlp.predict(d2_test_dataset)) # Test Accuracy

    acc_tr.append(a_tr)
    acc.append(a)
    timelog.append(endt)

In [ ]:
l = [10,20,50,100,200,500,1000]
N = len(l)
l2 = np.arange(N)
matplot.subplots(figsize=(10, 5))
matplot.plot(l2, acc, label="Testing Accuracy")
matplot.plot(l2, acc_tr, label="Training Accuracy")
matplot.xticks(l2,l)
matplot.grid(True)
matplot.xlabel("Hidden Layer Nodes")
matplot.ylabel("Accuracy")
matplot.legend()
matplot.title('Accuracy versus Nodes in the Hidden Layer for MLPClassifier', fontsize=12)
matplot.show()